<a href="https://colab.research.google.com/github/mudu89/ML_AssignmentPS01/blob/main/2024DA04200_Problem_Statement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/ML_Assignment_01/ML_AssignmentPS01

[Errno 2] No such file or directory: '/content/drive/MyDrive/ML_Assignment_01/ML_AssignmentPS01'
/content


### **1. Importing the dataset:**

Read the dataset from the path into a pandas dataframe.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from sklearn.preprocessing import StandardScaler
dataset_file = './risk_factors_cervical_cancer.csv'

df = pd.read_csv(dataset_file)

FileNotFoundError: [Errno 2] No such file or directory: './risk_factors_cervical_cancer.csv'

### **2. Data Exploration and Analysis:**

In this section, we will perform exploratory analysis of the dataset to get insight of the data, and understand the correlation of the variables.

We will also identify if the correlation analysis helps in the feature selection process.

In [ ]:
pd.set_option('display.max_columns', None)
print("The shape of the given dataset is:", df.shape)
print("Sanity Check:")
df.head(2)

In [ ]:
df.info()

We have a total of 35 attributes and 1 target variable in the dataset. There are mix of numeric and categorical data.

10 attributes are numeric and 26 are categorical.

We can group the columns into :
  
  a) Demographic data: Age, Number of sexual partners, First Sexual intercourse, number of pregnancies.

  b) Behavorial data: Smokes, Contraceptive usage.

  c) Medical history data: STDs, Cancer, CIN, HPV.
  
  d) Diagnosis data: Hinselmann, Schiller, Citology, Biopsy

**2.1. Univariate Analysis (Categorical Attributes):**

In this section, we will perform analysis of the categorical attributes to get more insights on their data.

In [ ]:
df.describe(exclude=['int64'])

From the above table, we can identify that majority of the attributes are having 3 or less unique values.

Also the frequency of the top value for such attributes is more than 70% of the total count.



In [ ]:
plt.figure(figsize=(4,3))
df['Biopsy'].value_counts(normalize=True).plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Biopsy Class Balance (0=No Cancer, 1=Cancer)')
plt.ylabel('Proportion')
plt.show()

There is a very high class imbalance on the target variable.

In [ ]:

fig, a = plt.subplots(15, 2, figsize=(18, 55))
fig.tight_layout(pad=6.0)
fig.align_titles()
fig.suptitle('Univariate Analysis of Categorical Attributes',y=1.0,va='top',fontsize=20)

axes = a.flatten()
gs = axes[0].get_gridspec()


sbplt_barh = 0
full_width_row = 11
for col in df.select_dtypes(include='object').columns:
  # Select the current axis
  if df[col].nunique() <= 25:
    kind = 'bar'
    ax = axes[sbplt_barh]
    sbplt_barh += 1
  else:
    kind = 'bar'
    axes[full_width_row * 2].remove()
    axes[full_width_row * 2 + 1].remove()
    ax = fig.add_subplot(gs[full_width_row, :])
    full_width_row += 1

  # # Use the selected axis
  df[col].value_counts(normalize=False).sort_values(ascending=True).plot(ax=ax, kind=kind, rot=0)
  ax.set_title(col) # Set title for clarity
plt.show()

The first sexual encounter chart shows that majority of the participants had their first encounter below the age of 16 and the number of partners chart indicates maximum participants had less than 3.

The bar chart for number of pregnancies shows that most women in the dataset
have between 1 and 3 pregnancies, with very few having more than 6.

The smoking status chart indicates that non-smokers form the majority, suggesting smoking is less common in this group.

The STDS, IUD and Hormanal contraceptives indicate high volume of 0 and '?' values.

**2.2: Univariate analysis of Numeric data:**

In this section, we will perform analysis of the numeric attributes to get more insights on their data.

In [ ]:
df.describe(exclude='object')

From the above table, we can see that the highest age is 84 and lowest is 13 with mean of 26. This indicates a spread of different age groups in the dataset.

In [ ]:
fig, a = plt.subplots(5, 5, figsize=(18, 18))
fig.tight_layout(pad=6.0)
fig.align_titles()
fig.suptitle('Univariate Analysis of Numeric Attributes',y=1.0,va='top',fontsize=20)

axes = a.flatten()
# gs = axes[0].get_gridspec()


# sbplt_barh = 0
# full_width_row = 11
for index, col in enumerate(df.select_dtypes(include='int64').columns):
  ax=axes[index]
  df[col].plot(kind='hist', ax=ax)
  ax.set_title(col) # Set title for clarity

num_of_attributes = (df.dtypes == 'int64').sum()
for ax in axes[num_of_attributes:]:
  fig.delaxes(ax)
plt.show()

The age histogram is right-skewed, with a concentration of women in their late 20s to early 40s and fewer older participants.  
Other numeric medical test results appear heavily zero-inflated, reflecting that many participants did not exhibit those conditions.

**2.3: Bivariate Analysis with the target variable:**

As seen in the univarte analysis, all the categorical attributes are having numbers as strings. Hence, we can convert all those attributes into numbers and perform a single correlation analysis using heatmap.

To perform correlation analysis using a heatmap, we need to convert all relevant columns to a numeric data type. The object type columns that contain '?' as a value need to be handled. I will replace '?' with NaN (Not a Number) and then convert these columns to a numeric type. NaN values are standard representations for missing data in pandas and can be handled appropriately in subsequent steps.

In [ ]:
df_copy = df.copy()

for col in df_copy.select_dtypes(include='object').columns:
    df_copy[col] = df_copy[col].replace('?', np.nan)
    df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

display(df_copy.info())

In [ ]:
# Correlation with target variable
corr_target = df_copy.corr(numeric_only=True)['Biopsy'].sort_values(ascending=False)
print("Correlation of features with Biopsy:")
print(corr_target)


In [ ]:
#plt.figure(figsize=(20, 15))
corr_matrix = df_copy.corr()
f, ax = plt.subplots(figsize=(20, 20))
f.suptitle('Correlation Heatmap',y=0.92,va='top',fontsize=20)
sns.heatmap(corr_matrix, annot=False, cmap='OrRd', ax=ax)
plt.show()

***Inference and Conclusion:***

Correlation helps spot strong linear relationships with the target.

Based on the heatmap and correlation analysis, we can identify correlation with the target variable as below:

*    Top correlated features with Biopsy are Schiller (0.73), Hinselmann (0.55), Citology (0.33)
*   Many STD-related features have weak correlations (below 0.15).
*   Age, Contraceptive choice, and sexual preferences seem to have very neglibile correlation with the target.
*   Several columns have NaN correlations due to constant values (e.g., STDs:AIDS always 0).

Effect on feature selection:

Strongly correlated features like Schiller, Hinselmann, and Citology are prime candidates to keep.

Weakly correlated ones may still hold value in non-linear models (like Decision Trees or Ensemble Methods), so we shouldn’t remove them solely based on correlation.

Features with almost no variance or with excessive missing values might be removed in preprocessing to reduce noise.

Strong correlation between independent variables themselves (multicollinearity) can reduce the stability of coefficients in linear models, even if each variable is useful individually.  
Example : STD's and STDs:Number of Diagnosis,
Dx: Cancer and Dx: HPV
  
**Conclusion:** Correlation analysis is a useful guide for identifying potentially important features and for spotting redundancy, but it should be combined with domain knowledge and other feature selection methods before making final decisions.

### **3. Data Preprocessing and Cleaning:**

In the previous section, we performed and identified the insights of the data attributes.

Based on the univariate analysis, we can now proceed to cleanup the data and perform preprocessing steps like:

a) NULL processing

b) Missing values cleanup

c) Managing skwed data

d) Identifying and processing outliers

e) Feature selection after data preprocessing. etc...

**3.1. Handling missing/null values and data transformation:**

From the univarite analysis, the categorical attributes are numbers and can be converted to numeric datatype.

We will replace the '?' values with NaN before converting all attributes to numeric.

We can handle the missing and NaN values by replacing them with the median value for the numeric attributes.

Justification: The median is chosen because it is less influenced by extreme values compared to the mean, which makes it more robust when data is skewed.


In [ ]:
# Replace '?' with NaN and transform objects to numeric datatype:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].replace('?', np.nan)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check for isnull/isna
print("Check for null values:")
print(df.isnull().sum())


In [ ]:
cols = df.columns
df[cols] = df[cols].fillna(df[cols].median())
df.sample(5, random_state=42)

In [ ]:
#Check for null after preprocessing:
print("Check for null values:")
print(df.isna().sum())

**3.2: Outlier Detection:**

Outliers are extreme or odd values lies far outside the general “pattern” or distribution in the dataset.

They distort/skew data resulting in less accuracy of the model.

We can identify outlier visually using boxplots/scatter plots and statistically using IQR method/z-score.


Justification: From the univariate analysis, only the below attributes are identified with variable values, whereas the rest of the attributes are either 0s or binary values (0,1). Hence selecting the below attributes for outlier analysis.
*   Age
*   Number of Sexual Partners
*   Number of Pregnancies
*   First Sexual encounter
*   Hormonal Contraceptives(Years)






In [ ]:
plt.figure(figsize=(15, 15))
#fig, a = plt.subplots(15, 2, figsize=(18, 55))
columns = ['Age', 'Number of sexual partners', 'First sexual intercourse','Num of pregnancies','Hormonal Contraceptives (years)']
for i, col in enumerate(columns, 1):
    plt.subplot(5, 1, i)
    plt.boxplot(df[col].dropna(), vert=False, patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='navy'),
            medianprops=dict(color='red', linewidth=2),
            whiskerprops=dict(color='navy'),
            capprops=dict(color='navy'),
            flierprops=dict(marker='o', color='darkorange', markersize=8))
    plt.title(col)

plt.suptitle("Outlier Detection in Numeric Features",y=0.98,va='top', fontsize=16)
plt.tight_layout(pad=3.0)
plt.show()

**Inference:**

Based on the boxplot charts, we can infer that there are outliers in the identified columns.

We are using capping to limit the values for 2 columns:

a) Number of sexual partners: 28 seems extreme and could be data entry anomaly.

b) First sexual intercourse: 32 is removed as it is rare.

**3.3: Skew Analysis:**



In [ ]:
skewness = df[columns].skew().sort_values(ascending=False)
print("Feature skewness:")
print(skewness)

From the skew analysis, we can identify that all the given attributes are highly skewed.

This matches what we saw in the outlier analysis, the skew is driven by those extreme high values.

We can perform transformation and capping to reduce the skewness.

In [ ]:
# #Capping the values for Number of sexual partners and first sexual partner
# capped_cols_values = {'Number of sexual partners': 15, 'First sexual intercourse': 10}

#First sexual intercourse
Q1 = df['First sexual intercourse'].quantile(0.25)
Q3 = df['First sexual intercourse'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
df['First sexual intercourse'] = np.where(df['First sexual intercourse'] < lower_bound, lower_bound, df['First sexual intercourse'])

# #Number of sexual partners
df['Number of sexual partners'] = np.where(df['Number of sexual partners'] > 15.0, 15.0, df['Number of sexual partners'])

#Hormonal Contraceptives (years)
df['Hormonal Contraceptives (years)'] = np.where(df['Hormonal Contraceptives (years)'] > 30.0, 3.0, df['Hormonal Contraceptives (years)'])

# Perform log transformation on columns:
for col in columns:
    if col != 'Biopsy':
      df[col] = np.log1p(df[col])
#df['Hormonal Contraceptives (years)'] = np.log1p(df['Hormonal Contraceptives (years)'])

skewness = df[columns].skew().sort_values(ascending=False)
print("Feature skewness after capping and transformation:")
print(skewness)

Post transformation, the skewness has reduced

**3.4: Standardisation or Normalization:**

In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns
num_cols = num_cols.drop('Biopsy')
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

**3.5: Feature Engineering:**

In [ ]:
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif

X = df.drop(columns=['Biopsy'])
y = df['Biopsy']


# ----- Step 1: Variance Threshold -----
selector = VarianceThreshold(threshold=0.01)
X_var = selector.fit_transform(X)
X_var_df = pd.DataFrame(X_var, columns=X.columns[selector.get_support()])

# ----- Step 2: Correlation Filtering -----
corr_matrix = X_var_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.85)]
X_corr = X_var_df.drop(columns=to_drop)

 # ----- Step 3: Mutual Information Ranking -----
mi = mutual_info_classif(X_corr, y, random_state=42)
mi_series = pd.Series(mi, index=X_corr.columns).sort_values(ascending=False)

    # Select top_k features
top_features = mi_series.head(10).index
#X_selected = X_corr[top_features]

# Plot feature importance
plt.figure(figsize=(10, 6))
mi_series.head(10).plot(kind='barh')
plt.title("Top 10 Important Features for Biopsy Prediction")
plt.xlabel("Feature Importance Score")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### **4. Model Building:**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

X = df[top_features]
y = df['Biopsy']  # Use original binary target

# Split data into train and test

(X_train, X_test, y_train, y_test) = train_test_split(X, y, stratify=y,test_size= 0.2)

# testing size = 20 %
# rest 80 % is used for training
# stratify parameter ensures that observations from each class is are given equal weightage

print(X_train.shape)
print(X_test.shape)

fit_result = {}

**4.1. Logistic Regression:**

In [ ]:

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}
grid_lr = GridSearchCV(LogisticRegression(class_weight='balanced', max_iter=500),
                       param_grid_lr, cv=5, scoring='f1')
grid_lr.fit(X_train, y_train)

lr_result_df = pd.DataFrame(grid_lr.cv_results_)


best_lr = grid_lr.best_estimator_
print(f"The best fit params selected are \n C:{best_lr.get_params()['C']}, \n solver:{best_lr.get_params()['solver']} ")
#logistic_model = LogisticRegression(solver='liblinear', class_weight='balanced') # default classfier

#logistic_model.fit(X_train,y_train)

# predict values for the test data
y_predict_logisticRegression  = best_lr.predict(X_test)

y_prob_logisticRegression = best_lr.predict_proba(X_test)[:, 1]

fit_result['Logistic Regression'] = [y_predict_logisticRegression , y_prob_logisticRegression]

**4.2. Decision Tree:**

In [ ]:
param_grid_dt = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}
grid_dt = GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=42),
                       param_grid_dt, cv=5, scoring='f1')
grid_dt.fit(X_train, y_train)
best_dt = grid_dt.best_estimator_

# predict values for the test data
y_predict_decisionTree  = best_dt.predict(X_test)

y_prob_decisionTree = best_dt.predict_proba(X_test)[:, 1]

fit_result['Decision Tree'] = [y_predict_decisionTree,  y_prob_decisionTree]

**4.3. K-Nearest Neighbour**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 11],
    'weights': ['uniform', 'distance']
}
grid_knn = GridSearchCV(KNeighborsClassifier(),
                        param_grid_knn, cv=5, scoring='f1')
grid_knn.fit(X_train, y_train)
best_knn = grid_knn.best_estimator_
# predict values for the test data
y_predict_knn  = best_knn.predict(X_test)

y_prob_knn = best_knn.predict_proba(X_test)[:, 1]

fit_result['KNN'] = [y_predict_knn, y_prob_knn]

**4.4. Ensemble - RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}
grid_rf = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42),
                       param_grid_rf, cv=5, scoring='f1')
grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_
# predict values for the test data
y_predict_rf  = best_rf.predict(X_test)

y_prob_rf = best_rf.predict_proba(X_test)[:, 1]

fit_result['Random Forest'] = [y_predict_rf,y_prob_rf]

### **5. Model Evaluation**

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
results = []
for model, model_result in fit_result.items():
    y_pred, y_prob = model_result
    results.append({
        "Model": model,
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_prob)
    })

results_df = pd.DataFrame(results)
results_df.head()



In [ ]:
# Create a 3x2 grid (6 slots). We'll use 5: 4 metrics + 1 ROC curve
fig, axes = plt.subplots(3, 2, figsize=(14, 16))
axes = axes.flatten()

metrics = ["Precision", "Recall", "F1-Score", "ROC-AUC"]

# Plot metric comparison bar charts
plt.suptitle('Model Evaluation metrics', y=1.0,va='top',fontsize=20)
for i, metric in enumerate(metrics):
    results_df.plot(
        x="Model", y=metric, kind="bar", ax=axes[i], legend=False, color="skyblue"
    )
    axes[i].set_title(f"{metric} Comparison")
    axes[i].set_ylabel(metric)
    axes[i].set_ylim(0, 1)

# ROC Curve (all models together in one subplot)
ax = axes[4]
for model, model_result in fit_result.items():
    y_pred, y_prob = model_result
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc_score = roc_auc_score(y_test, y_prob)
    ax.plot(fpr, tpr, label=f"{model} (AUC = {auc_score:.2f})")

ax.plot([0, 1], [0, 1], 'k--')
ax.set_title("ROC Curve Comparison")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate (Recall)")
ax.legend()

# Remove the unused last subplot (axes[5])
fig.delaxes(axes[5])


plt.tight_layout(pad=2.0)
plt.show()

### **5.1. Detailed Model Performance Analysis**

In this section, we will conduct a comprehensive evaluation of all trained models to identify the best performer for cervical cancer prediction.

**Key Evaluation Criteria:**
- **Recall (Sensitivity)**: Most critical metric for medical diagnosis - minimizing false negatives
- **Precision**: Important for resource allocation - avoiding excessive false positives  
- **F1-Score**: Balanced measure combining precision and recall
- **ROC-AUC**: Overall discriminative ability of the model

**Medical Context Consideration:**
In cancer detection, missing a positive case (false negative) is more dangerous than a false alarm (false positive). Therefore, we prioritize models with high recall while maintaining reasonable precision.

In [ ]:
# Enhanced Performance Evaluation with Additional Metrics
from sklearn.metrics import (confusion_matrix, classification_report,
                           accuracy_score, balanced_accuracy_score,
                           matthews_corrcoef, average_precision_score)
import pandas as pd
import numpy as np

# Create comprehensive results dataframe
enhanced_results = []

print("="*80)
print("COMPREHENSIVE MODEL PERFORMANCE EVALUATION")
print("="*80)

for model_name, model_result in fit_result.items():
    y_pred, y_prob = model_result

    # Calculate all metrics
    accuracy = accuracy_score(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    avg_precision = average_precision_score(y_test, y_prob)
    mcc = matthews_corrcoef(y_test, y_pred)

    # Confusion matrix components
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)

    enhanced_results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Balanced_Accuracy": balanced_acc,
        "Precision": precision,
        "Recall": recall,
        "Specificity": specificity,
        "F1_Score": f1,
        "ROC_AUC": roc_auc,
        "Avg_Precision": avg_precision,
        "MCC": mcc,
        "True_Positives": tp,
        "False_Positives": fp,
        "True_Negatives": tn,
        "False_Negatives": fn
    })

    print(f"\n{model_name} Performance:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  ROC-AUC: {roc_auc:.4f}")
    print(f"  Confusion Matrix: TP={tp}, FP={fp}, TN={tn}, FN={fn}")

# Create enhanced results dataframe
enhanced_results_df = pd.DataFrame(enhanced_results)
print("\n" + "="*80)
print("SUMMARY TABLE - ALL METRICS")
print("="*80)
print(enhanced_results_df.round(4))

In [ ]:
# Create comprehensive visualization for model comparison
plt.figure(figsize=(20, 15))

# Key metrics for medical diagnosis
key_metrics = ['Precision', 'Recall', 'F1_Score', 'ROC_AUC', 'Balanced_Accuracy', 'MCC']

# Create subplots for different metric comparisons
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Comprehensive Model Performance Comparison', fontsize=16, fontweight='bold')

# Plot individual metrics
for i, metric in enumerate(key_metrics):
    row = i // 3
    col = i % 3
    ax = axes[row, col]

    # Bar plot for each metric
    bars = ax.bar(enhanced_results_df['Model'], enhanced_results_df[metric],
                  color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'])

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

    ax.set_title(f'{metric.replace("_", " ")}', fontweight='bold')
    ax.set_ylabel('Score')
    ax.set_ylim(0, 1.1)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

# Confusion Matrix Heatmap
ax = axes[2, 0]
confusion_data = enhanced_results_df[['Model', 'True_Positives', 'False_Positives', 'True_Negatives', 'False_Negatives']]
cm_matrix = confusion_data.set_index('Model').T
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Confusion Matrix Components', fontweight='bold')

# ROC Curves Comparison
ax = axes[2, 1]
for model_name, model_result in fit_result.items():
    y_pred, y_prob = model_result
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc_score = roc_auc_score(y_test, y_prob)
    ax.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})', linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves Comparison', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Model Ranking Summary
ax = axes[2, 2]
# Create ranking based on key metrics (weighted by importance in medical context)
weights = {'Recall': 0.4, 'F1_Score': 0.3, 'ROC_AUC': 0.2, 'Precision': 0.1}
enhanced_results_df['Weighted_Score'] = sum(enhanced_results_df[metric] * weight
                                           for metric, weight in weights.items())

ranking = enhanced_results_df.sort_values('Weighted_Score', ascending=False)
bars = ax.barh(ranking['Model'], ranking['Weighted_Score'],
               color=['gold', 'silver', '#CD7F32', 'lightgray'])

# Add value labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2.,
            f'{width:.3f}', ha='left', va='center', fontweight='bold')

ax.set_xlabel('Weighted Performance Score')
ax.set_title('Overall Model Ranking\n(Weighted by Medical Importance)', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print ranking
print("\n" + "="*60)
print("MODEL RANKING (Weighted by Medical Importance)")
print("Weights: Recall=40%, F1=30%, ROC-AUC=20%, Precision=10%")
print("="*60)
for i, row in ranking.iterrows():
    print(f"{ranking.index.get_loc(i)+1}. {row['Model']}: {row['Weighted_Score']:.4f}")

In [ ]:
# Detailed Confusion Matrix Analysis for Each Model
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Detailed Confusion Matrix Analysis', fontsize=16, fontweight='bold')

model_names = list(fit_result.keys())
axes = axes.flatten()

for i, (model_name, model_result) in enumerate(fit_result.items()):
    y_pred, y_prob = model_result
    cm = confusion_matrix(y_test, y_pred)

    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['No Cancer', 'Cancer'],
                yticklabels=['No Cancer', 'Cancer'])

    axes[i].set_title(f'{model_name}\nConfusion Matrix', fontweight='bold')
    axes[i].set_xlabel('Predicted Label')
    axes[i].set_ylabel('True Label')

    # Add performance text
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Add text box with key metrics
    textstr = f'Sensitivity: {sensitivity:.3f}\nSpecificity: {specificity:.3f}\nFN (Missed): {fn}\nFP (False Alarm): {fp}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    axes[i].text(0.02, 0.98, textstr, transform=axes[i].transAxes, fontsize=9,
                verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

# Medical Impact Analysis
print("\n" + "="*80)
print("MEDICAL IMPACT ANALYSIS")
print("="*80)

for model_name, model_result in fit_result.items():
    y_pred, y_prob = model_result
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    print(f"\n{model_name}:")
    print(f"  • Correctly identified cancer cases (True Positives): {tp}")
    print(f"  • Missed cancer cases (False Negatives): {fn} ⚠️")
    print(f"  • False alarms (False Positives): {fp}")
    print(f"  • Correctly identified healthy cases (True Negatives): {tn}")

    if fn > 0:
        print(f"  • ⚠️  WARNING: {fn} cancer cases would be MISSED by this model!")
    if fp > 10:  # Arbitrary threshold
        print(f"  • ⚠️  NOTE: {fp} unnecessary follow-ups due to false alarms")

    # Calculate cost implications (hypothetical)
    missed_cost = fn * 10000  # High cost for missing cancer
    false_alarm_cost = fp * 500  # Lower cost for unnecessary tests
    total_cost = missed_cost + false_alarm_cost
    print(f"  • Estimated cost impact: ${total_cost:,} (${missed_cost:,} from missed cases + ${false_alarm_cost:,} from false alarms)")